In [ ]:
#!pip install deeppavlov
#!python -m deeppavlov install gobot_dstc2_minimal

## 0. Подготовка данных

Тут нужно создать папку assistant_data и поместить в нее файлы (четыре)
После этого можно запускать дальше
Считываем данные:

In [ ]:
from deeppavlov.dataset_readers.dstc2_reader import SimpleDSTC2DatasetReader


class AssistantDatasetReader(SimpleDSTC2DatasetReader):
    
    
    @staticmethod
    def _data_fname(datatype):
        assert datatype in ('val', 'trn', 'tst'), "wrong datatype name"
        return f"assistant-{datatype}.json"

In [ ]:
data = AssistantDatasetReader().read('assistant_data')

In [ ]:
!ls assistant_data

In [ ]:
!head -n 310 assistant_data/assistant-trn.json

Делаем итератор по данным


In [ ]:
from deeppavlov.dataset_iterators.dialog_iterator import DialogDatasetIterator

iterator = DialogDatasetIterator(data)

You can now iterate over batches of preprocessed dialogs:

In [ ]:
from pprint import pprint

for dialog in iterator.gen_batches(batch_size=1, data_type='train'):
    turns_x, turns_y = dialog
    
    print("User utterances:\n----------------\n")
    pprint(turns_x[0], indent=4)
    print("\nSystem responses:\n-----------------\n")
    pprint(turns_y[0], indent=4)
    
    break

print("\n-----------------")    
print(f"{len(iterator.get_instances('train')[0])} dialog(s) in train.")
print(f"{len(iterator.get_instances('valid')[0])} dialog(s) in valid.")
print(f"{len(iterator.get_instances('test')[0])} dialog(s) in test.")

## 1. Учим бота



&nbsp;
![gobot_simple_templates.png](https://github.com/deepmipt/DeepPavlov/blob/master/examples/img/gobot_simple_templates.png?raw=1)
&nbsp;

Шаблоны должны быть в формате `act` TAB `<template>` где `<act> `- это диалоговое действие, а `<template>` - соответствующий ответ.



Список действий для нашего бота

In [ ]:
!head -n 10 assistant_data/assistant-templates.txt


По сути, модуль диалога решает задачу классификации, где набор классов определяется в `assistant-templates.txt`. Таким образом, для обучения сети политики диалога вам понадобится ярлык действия для каждой очереди системы в обучающих диалогах. Наш набор данных помощника предоставляет `" act "` ключ словаря, который содержит действие, связанное с текущим ответом. Вот пример данных обучения для сети.

In [ ]:
!head -n 31 assistant_data/assistant-trn.json

Навыки в DeepPavlov определяются конфигурационными файлами. Итак, мы будем использовать минимальную конфигурацию бота DSTC2 (доступно больше конфигураций) и поменять разделы, отвечающие за:
- embeddings,
- шаблоны ответов,
- пути загрузки и сохранения данных и моделей.

Загрузим бот:

In [ ]:
from deeppavlov import configs
from deeppavlov.core.common.file import read_json

gobot_config = read_json(configs.go_bot.gobot_dstc2_minimal)

In [ ]:
from deeppavlov.download import download_resource

#download_resource(url="http://files.deeppavlov.ai/embeddings/glove.6B.100d.txt",
#                  dest_paths=['assistant_bot/'])

Изменяем конфигурации


In [ ]:
gobot_config['chainer']['pipe'][-1]['embedder'] = {
    "class_name": "glove",
    "load_path": "assistant_bot/small.txt"
}

In [ ]:
gobot_config['chainer']['pipe'][-1]['nlg_manager']['template_path'] = 'assistant_data/assistant-templates.txt'
gobot_config['chainer']['pipe'][-1]['nlg_manager']['api_call_action'] = None

In [ ]:
gobot_config['dataset_reader']['class_name'] = '__main__:AssistantDatasetReader'
gobot_config['metadata']['variables']['DATA_PATH'] = 'assistant_data'

gobot_config['metadata']['variables']['MODEL_PATH'] = 'assistant_bot'

In [ ]:
from deeppavlov import train_model

gobot_config['train']['batch_size'] = 4 # set batch size
gobot_config['train']['max_batches'] = 30 # maximum number of training batches
gobot_config['train']['val_every_n_batches'] = 30 # evaluate on full 'valid' split every 30 epochs
gobot_config['train']['log_every_n_batches'] = 5 # evaluate on full 'train' split every 5 batches

train_model(gobot_config);



Можно поменять конфиг файл [config doc page](http://docs.deeppavlov.ai/en/master/intro/configuration.html) 

# 2. Посмотрим как работает бот и запустим его в телеграмме 
Библиотека предоставляет возможности так же запускать на других платформах

In [ ]:
from deeppavlov import build_model

bot = build_model(gobot_config)

In [ ]:
bot([[{"text": "Добрый день"}]])

In [ ]:
bot([[{"text": "монета"}]])

In [ ]:
bot([[{"text": "золотые"}]])

In [ ]:
bot([[{"text": "пока"}]])

In [ ]:
bot.reset()

In [ ]:
bot([[{"text": "Привет"}]])

In [ ]:
bot([[{"text": "Кредит"}]])

In [ ]:
bot([[{"text": "Коммерческий"}]])

In [ ]:
bot([[{"text": "Пока"}]])

In [ ]:
bot.reset()

Посмотрим как это выглядит в телеграмме

In [ ]:
from deeppavlov.utils.telegram import interact_model_by_telegram

interact_model_by_telegram(model_config=gobot_config, token='1153548935:AAFIZkbBaYKjzlpum6wVM6oTHviL4VYlPY8')